## 准备数据

In [18]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [19]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [20]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28*28+1, 100]), name='W1')
        self.b1 = tf.Variable(tf.zeros([1, 100]), name='b1')
        self.W2 = tf.Variable(tf.random.normal([100, 10]), name='W2')
        self.b2 = tf.Variable(tf.zeros([1, 10]), name='b2')

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 28*28))
        bias = tf.ones([x.shape[0], 1])
        x = tf.concat([x, bias], axis=1)

        h1 = tf.matmul(x, self.W1) + self.b1
        h1_relu = tf.nn.relu(h1)
        h2 = tf.matmul(h1_relu, self.W2) + self.b2
        logits = tf.nn.log_softmax(h2)
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [21]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    # for g, v in zip(grads, trainable_vars):
    #     v.assign_sub(0.01*g)
    optimizer.apply_gradients(zip(grads, trainable_vars))

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [28]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 5.376735 ; accuracy 0.7920333
epoch 1 : loss 5.358925 ; accuracy 0.79263335
epoch 2 : loss 5.341231 ; accuracy 0.79305
epoch 3 : loss 5.323648 ; accuracy 0.79366666
epoch 4 : loss 5.3061852 ; accuracy 0.7942333
epoch 5 : loss 5.2888293 ; accuracy 0.79473335
epoch 6 : loss 5.2715883 ; accuracy 0.79538333
epoch 7 : loss 5.254459 ; accuracy 0.79586667
epoch 8 : loss 5.2374396 ; accuracy 0.7962833
epoch 9 : loss 5.220537 ; accuracy 0.7966667
epoch 10 : loss 5.203751 ; accuracy 0.79711664
epoch 11 : loss 5.187079 ; accuracy 0.79766667
epoch 12 : loss 5.1705146 ; accuracy 0.7982
epoch 13 : loss 5.1540594 ; accuracy 0.79871666
epoch 14 : loss 5.137708 ; accuracy 0.79925
epoch 15 : loss 5.12147 ; accuracy 0.7996333
epoch 16 : loss 5.1053295 ; accuracy 0.8001
epoch 17 : loss 5.0892854 ; accuracy 0.80055
epoch 18 : loss 5.073342 ; accuracy 0.80093336
epoch 19 : loss 5.057499 ; accuracy 0.80128336
epoch 20 : loss 5.0417514 ; accuracy 0.80181664
epoch 21 : loss 5.026108 ; accuracy 0